In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/notebooks/fabian/randomforestpoutrespipeline.pkl"
pipeline = joblib.load(pipeline_path)

In [3]:
# Charger le CSV
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data/datatest/test/poutres1.csv"
df_test = pd.read_csv(csv_path, delimiter=';')

# Nettoyer les noms de colonnes comme à l'entraînement
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

df_test.columns = [clean_col(c) for c in df_test.columns]

In [4]:
df_test

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,tableau_1
Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,AI,AS,Hauteur totale,Hauteur,Sols en intersection,Sols coupés (u),Sols coupés (Ids),Sols coupants (u),Sols coupants (Ids),Sol au-dessus,Sol en-dessous,Poteaux en intersection,Poteaux coupés (u),Poteauc coupés (Ids),Poteaux coupants (u),Poteaux coupants (Ids),Image,Etat de la jonction,Valeur de décalage Z,Justification Z,Valeur de décalage Y,Justification Y,Justification YZ,Catégorie,Type prédéfini d'IFC,Exporter au format IFC sous,Exporter au format IFC,IfcGUID,A une association,Enrobage d'armature - Face inférieure,Enrobage d'armature - Face supérieure,Enrobage d'armature - Autres faces,Variantes,Volume,Phase de démolition,Phase de création,Commentaires,Matériau structurel,ID hôte,Niveau,Famille et type,Famille,Type,Nom de la famille,Nom du type,ID du type,Elévation du niveau de référence,Elévation en haut,Rotation de la section,Orientation,Décalage du niveau d'arrivée,Décalage du niveau de départ,Elévation à la base,Longueur de coupe,Niveau de référence,Utilisation structurelle,Plan de construction,Longueur,Identifiant,Visible dans les nomenclatures,Légende détails,Composition,IdGtcParameter,IdGtcParameterCreationTime,Image du type,Note d'identification,Type: Type prédéfini d'IFC,Exporter le type au format IFC sous,Exporter le type au format IFC,Type IfcGUID,Modèle,Fabricant,Commentaires du type,URL,Description,Nom de code,Identifiant du nom de la coupe,Forme de coupe,Titre OmniClass,Numéro OmniClass,Description de l'assemblage,Code d'assemblage,Marque de type,Protection contre l'incendie,Coût,T - Dimension,ECH_hauteur,ECH_largeur,Type d'attachement de fin,Type d'attachement de début,STR_NIV_ref,Prefixe,Classe Beton,STR_As NGF,STR_Ai NGF,HAUTEUR petite section,LONGUEUR petite section,LARGEUR,HAUTEUR
687399,NaN,NaN,NaN,NaN,20x20ht,"2,79999999999975","3,04999999999976","0,2","0,250000000000005",0,0,NaN,1,637182,False,True,0,0,NaN,0,NaN,<Aucun>,0,"-0,05",0,0,2,0,Ossature,NaN,NaN,0,08OgeSaanE5vbhv6OOZZY3,1,NaN,NaN,NaN,-1,0,Aucun(e),Nouvelle construction,NaN,EIF_STR - BETON,-1,NaN,EIF_STR - Poutre: 20x20ht,EIF_STR - Poutre,20x20ht,NaN,NaN,1189537,"3,04999999999976","2,99999999999976",0,0,0,0,"2,79999999999976","2,54",A&B_R+1,6,Niveau : A&B_R+1,"2,54",BN,0,NaN,NaN,0,0,<Aucun>,NaN,NaN,NaN,0,3plq2udQjCFAFkt6ORWZvT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,NaN,"0,2","0,2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
694648,NaN,NaN,NaN,NaN,20x20ht,"2,79999999999975","3,04999999999976","0,2","0,250000000000005",0,0,NaN,1,637182,False,True,0,0,NaN,0,NaN,<Aucun>,0,"-0,05",0,0,2,0,Ossature,NaN,NaN,0,08OgeSaanE5vbhv6OOZjpS,1,NaN,NaN,NaN,-1,0,Aucun(e),Nouvelle construction,NaN,EIF_STR - BETON,-1,NaN,EIF_STR - Poutre: 20x20ht,EIF_STR - Poutre,20x20ht,NaN,NaN,1189537,"3,04999999999976","2,99999999999976",0,0,"0,000000000000000541433564649196","0,000000000000000541433564649196","2,79999999999976","1,12",A&B_R+1,6,Niveau : A&B_R+1,"1,12",BN,0,NaN,NaN,0,0,<Aucun>,NaN,NaN,NaN,0,3plq2udQjCFAFkt6ORWZvT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,NaN,"0,2","0,2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
696921,NaN,NaN,NaN,NaN,16x20ht,"2,79999999999975","3,04999999999976","0,2","0,250000000000005",0,0,NaN,2,"637182,694812",False,True,0,0,NaN,0,NaN,<Aucun>,0,"-0,05",0,0,2,0,Ossature,NaN,NaN,0,08OgeSaanE5vbhv6OOZkVz,1,NaN,NaN,NaN,-1,0,Aucun(e),Nouvelle construction,NaN,EIF_STR - BETON,-1,NaN,EIF_STR - Poutre: 16x20ht,EIF_STR - Poutre,16x20ht,NaN,NaN,1189921,"3,04999999999976","2,99999999999976",0,0,"0,000000000000000541433564649196","0,000000000000000541433564649196","2,79999999999976","2,62",A&B_R+1,6,Niveau : A&B_R+1,"2,62",BN,0,NaN,NaN,0,0,<Aucun>,NaN,NaN,NaN,0,3plq2udQjCFAFkt6ORWZ3T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Beams,23.25.30.11.14.14,NaN,NaN,NaN,NaN,0,NaN,"0,2","0,16",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Colonnes explicatives attendues par le pipeline
features = pipeline.named_steps['preprocessor'].feature_names_in_

# S'assurer que toutes les colonnes sont présentes
for col in features:
    if col not in df_test.columns:
        df_test[col] = np.nan
X_test = df_test[features].copy()

In [6]:
y_pred = pipeline.predict(X_test)

In [7]:
y_pred

array([['GO', 'POUTRE', 'COURANT', 'COULE EN PLACE'],
       ['GO', 'POUTRE', 'COURANT', 'COULE EN PLACE'],
       ['GO', 'POUTRE', 'COURANT', 'COULE EN PLACE'],
       ...,
       ['GO', 'POUTRE', 'COURANT', 'COULE EN PLACE'],
       ['GO', 'POUTRE', 'COURANT', 'COULE EN PLACE'],
       ['GO', 'POUTRE', 'COURANT', 'COULE EN PLACE']], dtype=object)

In [8]:
# Les targets à prédire (après nettoyage)
targets = [
    "011ec_lot",
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred, columns=targets)

# Afficher les premières lignes
print(df_pred.head(50))

   011ec_lot 012ec_ouvrage 013ec_localisation 014ec_mode_constructif
0         GO        POUTRE            COURANT         COULE EN PLACE
1         GO        POUTRE            COURANT         COULE EN PLACE
2         GO        POUTRE            COURANT         COULE EN PLACE
3         GO        POUTRE            COURANT         COULE EN PLACE
4         GO        POUTRE            COURANT         COULE EN PLACE
5         GO        POUTRE            COURANT         COULE EN PLACE
6         GO        POUTRE            COURANT         COULE EN PLACE
7         GO        POUTRE            COURANT         COULE EN PLACE
8         GO        POUTRE            COURANT         COULE EN PLACE
9         GO        POUTRE            COURANT         COULE EN PLACE
10        GO        POUTRE            COURANT         COULE EN PLACE
11        GO        POUTRE            COURANT         COULE EN PLACE
12        GO        POUTRE            COURANT         COULE EN PLACE
13        GO        POUTRE        

In [ ]:
df_pred.to_csv("resultats_predictionspoutres.csv", index=False)